# Day 14: Parabolic Reflector Dish

You reach the place where all of the mirrors were pointing: a massive parabolic reflector dish attached to the side of another large mountain.

The dish is made up of many small mirrors, but while the mirrors themselves are roughly in the shape of a parabolic reflector dish, each individual mirror seems to be pointing in slightly the wrong direction. If the dish is meant to focus light, all it's doing right now is sending it in a vague direction.

This system must be what provides the energy for the lava! If you focus the reflector dish, maybe you can go where it's pointing and use the light to fix the lava production.

Upon closer inspection, the individual mirrors each appear to be connected via an elaborate system of ropes and pulleys to a large metal platform below the dish. The platform is covered in large rocks of various shapes. Depending on their position, the weight of the rocks deforms the platform, and the shape of the platform controls which ropes move and ultimately the focus of the dish.

In short: if you move the rocks, you can focus the dish. The platform even has a control panel on the side that lets you tilt it in one of four directions! The rounded rocks (O) will roll when the platform is tilted, while the cube-shaped rocks (#) will stay in place. You note the positions of all of the empty spaces (.) and rocks (your puzzle input). For example:

```
O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....
```

Start by tilting the lever so all of the rocks will slide north as far as they will go:

```
OOOO.#.O..
OO..#....#
OO..O##..O
O..#.OO...
........#.
..#....#.#
..O..#.O.O
..O.......
#....###..
#....#....
```

You notice that the support beams along the north side of the platform are damaged; to ensure the platform doesn't collapse, you should calculate the total load on the north support beams.

The amount of load caused by a single rounded rock (O) is equal to the number of rows from the rock to the south edge of the platform, including the row the rock is on. (Cube-shaped rocks (#) don't contribute to load.) So, the amount of load caused by each rock in each row is as follows:

```
OOOO.#.O.. 10
OO..#....#  9
OO..O##..O  8
O..#.OO...  7
........#.  6
..#....#.#  5
..O..#.O.O  4
..O.......  3
#....###..  2
#....#....  1
```

The total load is the sum of the load caused by all of the rounded rocks. In this example, the total load is 136.

Tilt the platform so that the rounded rocks all roll north. Afterward, what is the total load on the north support beams?

In [1]:
from typing import List

# Loading input file
platform: List[List[str]] = []
with open("2023_day_14_input.txt") as f:
    for line in f:
        platform.append([c for c in line if c != "\n"])

# Util functions
def show(platform: List[List[str]]):
    """Nicely print the platform `platform`
    """
    print("\n".join(["".join(l) for l in platform]))

def north_load(platform: List[List[str]]) -> int:
    """Compute the total load on the north support beam
    """
    rows = len(platform)
    res = 0
    for i, line in enumerate(platform):
        load = rows - i
        for c in line:
            if c != "O":
                continue
            res += load
    return res

def tilt_platform(platform: List[List[str]]) -> List[List[str]]:
    """Return the result of tilting the platform in one direction"""

    res = [r.copy() for r in platform]

    # Get platform dimensions
    rows = len(platform)
    cols = len(platform[0])

    # Keep track of the last free spot for a rolling object
    last_free_row= [0] * cols

    for i, line in enumerate(platform):
        for j, c in enumerate(line):
            if c == "#":
                # If square rock the next free sport is necessarily after
                last_free_row[j] = i + 1
            if c == "O":
                if last_free_row[j] != i:
                    # In that case the rolling rock can roll to a different position
                    res[last_free_row[j]][j] = "O"
                    res[i][j] = "."
                # In all cases the next free spot is increased
                last_free_row[j] = last_free_row[j] + 1
    
    return res

# Testing basic scenarios
assert tilt_platform([["O", "O"], ["O", "O"]]) == [["O", "O"], ["O", "O"]]
assert tilt_platform([[".", "#"], ["O", "O"]]) == [["O", "#"], [".", "O"]]
assert tilt_platform([[".", "#"], ["#", "."]]) == [[".", "#"], ["#", "."]]

tilted_platform = tilt_platform(platform)
north_load(tilted_platform)
    

108918

# Part Two

The parabolic reflector dish deforms, but not in a way that focuses the beam. To do that, you'll need to move the rocks to the edges of the platform. Fortunately, a button on the side of the control panel labeled "spin cycle" attempts to do just that!

Each cycle tilts the platform four times so that the rounded rocks roll north, then west, then south, then east. After each tilt, the rounded rocks roll as far as they can before the platform tilts in the next direction. After one cycle, the platform will have finished rolling the rounded rocks in those four directions in that order.

Here's what happens in the example above after each of the first few cycles:

After 1 cycle:

```
.....#....
....#...O#
...OO##...
.OO#......
.....OOO#.
.O#...O#.#
....O#....
......OOOO
#...O###..
#..OO#....
```

After 2 cycles:

```
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#..OO###..
#.OOO#...O
```

After 3 cycles:

```
.....#....
....#...O#
.....##...
..O#......
.....OOO#.
.O#...O#.#
....O#...O
.......OOO
#...O###.O
#.OOO#...O
```

This process should work if you leave it running long enough, but you're still worried about the north support beams. To make sure they'll survive for a while, you need to calculate the total load on the north support beams after 1000000000 cycles.

In the above example, after 1000000000 cycles, the total load on the north support beams is 64.

Run the spin cycle for 1000000000 cycles. Afterward, what is the total load on the north support beams?

In [78]:
from typing import List
from itertools import product, count

# Loading input file
platform: List[List[str]] = []
with open("2023_day_14_input.txt") as f:
    for line in f:
        platform.append([c for c in line if c != "\n"])

# Util functions
def show(platform: List[List[str]]):
    """Nicely print the platform `platform`
    """
    print("\n".join(["".join(l) for l in platform]))

def north_load(platform: List[List[str]]) -> int:
    """Compute the total load on the north support beam
    """
    rows = len(platform)
    res = 0
    for i, line in enumerate(platform):
        load = rows - i
        for c in line:
            if c != "O":
                continue
            res += load
    return res

def tilt_platform(platform: List[List[str]], orient: str) -> List[List[str]]:
    """Return the result of tilting the platform in one direction (`orient`)
    
    direction can have 4 values: "N", "W", "S" and "E"
    """
    assert orient in ["N", "W", "S", "E"]

    res = [r.copy() for r in platform]

    # Get platform dimensions
    rows = len(platform)
    cols = len(platform[0])

    iterator = None
    if orient == "N":
        iterator = product(range(rows), range(cols))
    elif orient == "E":
        iterator = product(range(cols - 1, -1, -1), range(rows))
    elif orient == "S":
        iterator = product(range(rows - 1, -1, -1), range(cols))
    elif orient == "W":
        iterator = product(range(cols), range(rows))

    # Keep track of the last free spot for a rolling object
    last_free_row= [(0 if orient in ["N", "W"] else (rows - 1 if orient == "S" else cols - 1))] * (cols if orient in ["N", "S"] else rows)

    for i, j in iterator:
        pos = (i, j) if orient in ["N", "S"] else (j, i)
        scan_pos = j
        c = res[pos[0]][pos[1]]
        if c == "#":
            # If square rock the next free sport is necessarily after
            last_free_row[scan_pos] = (pos[0] if orient in ["N", "S"] else pos[1]) + (1 if orient in ["N", "W"] else -1)
        if c == "O":
            if last_free_row[scan_pos] != (pos[0] if orient in ["N", "S"] else pos[1]):
                # In that case the rolling rock can roll to a different position
                new_pos = (last_free_row[scan_pos], pos[1]) if orient in ["N", "S"] else (pos[0], last_free_row[scan_pos])
                res[new_pos[0]][new_pos[1]] = "O"
                res[pos[0]][pos[1]] = "."
            # In all cases the next free spot is increased
            last_free_row[scan_pos] = last_free_row[scan_pos] +  (1 if orient in ["N", "W"] else -1)
    
    return res

# Testing basic scenarios
assert tilt_platform([["O", "O"], ["O", "O"]], "N") == [["O", "O"], ["O", "O"]]
assert tilt_platform([[".", "#"], ["O", "O"]], "N") == [["O", "#"], [".", "O"]]
assert tilt_platform([[".", "#"], ["#", "."]], "N") == [[".", "#"], ["#", "."]]
assert tilt_platform([["O", "O"], ["O", "O"]], "S") == [["O", "O"], ["O", "O"]]
assert tilt_platform([["O", "O"], [".", "#"]], "S") == [[".", "O"], ["O", "#"]]
assert tilt_platform([[".", "#"], ["#", "."]], "S") == [[".", "#"], ["#", "."]]
assert tilt_platform([["O", "O"], ["O", "O"]], "E") == [["O", "O"], ["O", "O"]]
assert tilt_platform([["O", "."], ["O", "#"]], "E") == [[".", "O"], ["O", "#"]]
assert tilt_platform([[".", "#"], ["#", "."]], "E") == [[".", "#"], ["#", "."]]
assert tilt_platform([["O", "O"], ["O", "O"]], "W") == [["O", "O"], ["O", "O"]]
assert tilt_platform([[".", "O"], ["#", "O"]], "W") == [["O", "."], ["#", "O"]]
assert tilt_platform([[".", "#"], ["#", "."]], "W") == [[".", "#"], ["#", "."]]


def cycle_platform(platform: List[List[str]]) -> List[List[str]]:
    """Return the result of tilting the platform in all four direction
    
    The platform is titled in this with this sequence: "N", "W", "S", "E"
    """
    res = platform
    res = tilt_platform(res, "N")
    res = tilt_platform(res, "W")
    res = tilt_platform(res, "S")
    res = tilt_platform(res, "E")
    return res


assert sum([l1 != l2 for l1, l2 in zip(cycle_platform([list(r) for r in "O....#....,O.OO#....#,.....##...,OO.#O....O,.O.....O#.,O.#..O.#.#,..O..#O..O,.......O..,#....###..,#OO..#....".split(",")]), [list(r) for r in ".....#....,....#...O#,...OO##...,.OO#......,.....OOO#.,.O#...O#.#,....O#....,......OOOO,#...O###..,#..OO#....".split(",")])]) == 0


current_platform = [r.copy() for r in platform]
visited_states = [",".join(["".join(l) for l in platform])]
stable = False
i = 0
while not stable:
    res = cycle_platform(current_platform)
    res_repr = ",".join(["".join(l) for l in res])
    if res_repr in visited_states:
        stable = True
    current_platform = res
    visited_states.append(res_repr)
    i += 1

cycle_start = visited_states.index(visited_states[-1])
cycle_size = len(visited_states) - cycle_start - 1

resulting_state = visited_states[cycle_start + (1000000000 - cycle_start) % cycle_size]

north_load([list(r) for r in resulting_state.split(",")])

100310